In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")

In [4]:
del stats["Unnamed: 0"]

In [6]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          61
3P            0
3PA           0
3P%        2102
2P            0
2PA           0
2P%         105
eFG%         61
FT            0
FTA           0
FT%         545
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W             0
L             0
W/L%          0
GB            0
PS/G          0
PA/G          0
SRS           0
dtype: int64

In [7]:
stats = stats.fillna(0)

In [8]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

In [9]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS']

In [10]:
train = stats[stats["Year"] < 2023]

In [11]:
test = stats[stats["Year"] == 2023]

In [13]:
!pip3 install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.1 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 60.6 MB/s eta 0:00:00 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 34.3 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [15]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [16]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [17]:
predictions = reg.predict(test[predictors])

In [20]:
predictions = pd.DataFrame(predictions, columns=["predictions"], index = test.index)

In [21]:
predictions

,predictions
506,-0.018593
507,-0.025551
508,-0.001432
509,-0.011236
510,0.028890
...,...
15231,0.004033
15232,-0.005344
15233,-0.015625
15234,-0.004994


In [22]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [23]:
combination

,Player,Share,predictions
506,AJ Griffin,0.0,-0.018593
507,Aaron Holiday,0.0,-0.025551
508,Bogdan Bogdanović,0.0,-0.001432
509,Bruno Fernando,0.0,-0.011236
510,Clint Capela,0.0,0.028890
...,...,...,...
15231,Monte Morris,0.0,0.004033
15232,Quenton Jackson,0.0,-0.005344
15233,Taj Gibson,0.0,-0.015625
15234,Vernon Carey Jr.,0.0,-0.004994


In [24]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
11333,Joel Embiid,0.915,0.205298
3821,Nikola Jokić,0.674,0.180016
8327,Giannis Antetokounmpo,0.606,0.232607
1031,Jayson Tatum,0.280,0.144385
10296,Shai Gilgeous-Alexander,0.046,0.150429
2763,Donovan Mitchell,0.030,0.095006
12843,Domantas Sabonis,0.027,0.095971
3302,Luka Dončić,0.010,0.205332
4855,Stephen Curry,0.005,0.120301
7820,Jimmy Butler,0.003,0.109974


In [25]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combination["Share"], combination["predictions"])

0.002750988964163857

In [26]:
combination = combination.sort_values("Share", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0]+1))

In [27]:
combination.head(10)

,Player,Share,predictions,Rk
11333,Joel Embiid,0.915,0.205298,1
3821,Nikola Jokić,0.674,0.180016,2
8327,Giannis Antetokounmpo,0.606,0.232607,3
1031,Jayson Tatum,0.280,0.144385,4
10296,Shai Gilgeous-Alexander,0.046,0.150429,5
2763,Donovan Mitchell,0.030,0.095006,6
12843,Domantas Sabonis,0.027,0.095971,7
3302,Luka Dončić,0.010,0.205332,8
4855,Stephen Curry,0.005,0.120301,9
7820,Jimmy Butler,0.003,0.109974,10


In [28]:
combination = combination.sort_values("predictions", ascending=False)
combination["Predicted Rk"] = list(range(1, combination.shape[0]+1))

In [29]:
combination.head(10)

,Player,Share,predictions,Rk,Predicted Rk
8327,Giannis Antetokounmpo,0.606,0.232607,3,1
3302,Luka Dončić,0.010,0.205332,8,2
11333,Joel Embiid,0.915,0.205298,1,3
3821,Nikola Jokić,0.674,0.180016,2,4
10296,Shai Gilgeous-Alexander,0.046,0.150429,5,5
11839,Kevin Durant,0.000,0.147943,234,6
12332,Damian Lillard,0.000,0.145348,225,7
1031,Jayson Tatum,0.280,0.144385,4,8
6913,Anthony Davis,0.000,0.143710,415,9
6920,LeBron James,0.000,0.143608,441,10


In [32]:
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row["Player"] in actual ["Player"].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps) / len(ps)

In [33]:
find_ap(combination)

0.7683333333333333

In [34]:
years = list(range(1991,2024))

In [37]:
aps = []
all_predictions = []
for year in years[5:]:
    train = stats[stats["Year"] < year]
    test = stats[stats["Year"] == year]
    reg.fit(train[predictors], train["Share"])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=["predictions"], index = test.index)
    combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
    all_predictions.append(combination)
    aps.append(find_ap(combination))

In [38]:
sum(aps) / len(aps)

0.7195472452665953

In [40]:
def add_ranks(combination):
    combination = combination.sort_values("Share", ascending=False)
    combination["Rk"] = list(range(1, combination.shape[0]+1))
    combination = combination.sort_values("predictions", ascending=False)
    combination["Predicted Rk"] = list(range(1, combination.shape[0]+1))
    combination["Diff"] = combination["Rk"] - combination["Predicted Rk"]
    return combination

In [41]:
add_ranks(all_predictions[1])

,Player,Share,predictions,Rk,Predicted Rk,Diff
6514,Shaquille O'Neal,0.006,0.198202,9,1,8
14194,Karl Malone,0.857,0.192318,1,2,-1
1820,Michael Jordan,0.832,0.167629,2,3,-1
4951,Hakeem Olajuwon,0.083,0.136357,7,4,3
12949,David Robinson,0.000,0.131089,179,5,174
...,...,...,...,...,...,...
8930,Eric Montross,0.000,-0.043824,78,437,-359
630,Pervis Ellison,0.000,-0.047037,308,438,-130
13666,Popeye Jones,0.000,-0.048904,138,439,-301
1814,Dennis Rodman,0.000,-0.054609,318,440,-122


In [42]:
def backtest(stats, model, year, predictors):
    aps = []
    all_predictions = []
    for year in years[5:]:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]
        model.fit(train[predictors], train["Share"])
        predictions = reg.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=["predictions"], index = test.index)
        combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps) / len(aps), aps, pd.concat(all_predictions)

In [44]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [45]:
mean_ap

0.7195472452665953

In [70]:
all_predictions[all_predictions["Rk"] <= 5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted Rk,Diff
9018,Jason Kidd,0.712,0.028368,2,51,-49
11555,Steve Nash,0.839,0.032278,1,48,-47
1493,Glen Rice,0.117,0.038456,5,47,-42
12546,Peja Stojaković,0.228,0.035470,4,39,-35
11573,Steve Nash,0.739,0.054378,1,34,-33
2073,Joakim Noah,0.258,0.046777,4,37,-33
4050,Chauncey Billups,0.344,0.052760,5,35,-30
11793,Chris Paul,0.138,0.072842,5,33,-28
11588,Steve Nash,0.785,0.074657,2,23,-21
11463,Jason Kidd,0.135,0.052195,5,21,-16


In [71]:
stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())

/var/folders/bf/l4h08ncx68xbswrxct3ky9w00000gn/T/ipykernel_62175/2022889540.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())


In [72]:
stat_ratios

PTS       AST       STL       BLK        3P  Year
Year                                                              
1991 0      1.692601  2.010078  2.608773  1.346939  5.594452   1.0
     1      2.884104  1.542618  2.059558  1.795918  5.085865   1.0
     2      0.679268  0.327222  0.549215  0.673469  0.000000   1.0
     3      0.055678  0.000000  0.000000  0.673469  0.000000   1.0
     4      1.514433  1.262142  0.823823  0.224490  1.017173   1.0
...              ...       ...       ...       ...       ...   ...
2023 15231  1.129221  2.561833  1.148205  0.523047  1.312418   1.0
     15232  0.679725  0.821720  0.656117  0.261524  0.100955   1.0
     15233  0.372752  0.338355  0.492088  0.523047  0.201910   1.0
     15234  0.054817  0.145009  0.328058  0.523047  0.000000   1.0
     15235  0.416606  0.290019  0.984175  1.046094  0.000000   1.0

[15236 rows x 6 columns]

In [73]:
stat_ratios = stat_ratios.reset_index(drop=True)

In [74]:
stats[["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

In [75]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,W/L%,GB,PS/G,PA/G,SRS,PTS_T,AST_R,STL_R,BLK_R,3P_R
0,Doc Rivers,PG,29,ATL,79,79,32.7,5.6,12.9,0.435,...,0.524,18.0,109.8,109.0,0.72,1.692601,2.010078,2.608773,1.346939,5.594452
1,Dominique Wilkins,SF,31,ATL,81,81,38.0,9.5,20.2,0.470,...,0.524,18.0,109.8,109.0,0.72,2.884104,1.542618,2.059558,1.795918,5.085865
2,Duane Ferrell,SF,25,ATL,78,2,14.9,2.2,4.6,0.489,...,0.524,18.0,109.8,109.0,0.72,0.679268,0.327222,0.549215,0.673469,0.000000
3,Gary Leonard,C,23,ATL,4,0,2.3,0.0,0.0,0.000,...,0.524,18.0,109.8,109.0,0.72,0.055678,0.000000,0.000000,0.673469,0.000000
4,John Battle,SG,28,ATL,79,2,23.6,5.0,10.9,0.461,...,0.524,18.0,109.8,109.0,0.72,1.514433,1.262142,0.823823,0.224490,1.017173


In [76]:
predictors += ["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [77]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [78]:
mean_aps

0.7195472452665953

In [79]:
stats["NPos"] = stats["Pos"].astype("category").cat.codes

In [80]:
stats["NTm"] = stats["Tm"].astype("category").cat.codes

In [82]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)

mean_ap, aps, all_predictions = backtest(stats, rf, years[28:], predictors)

In [83]:
mean_ap

0.7455287368553618

In [84]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[28:], predictors)

In [85]:
mean_ap

0.7128341606631725